In [5]:
import json
import pandas as pd
import csv
import sklearn as skl
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk import ngrams
import random
import string
import Levenshtein as lv
import nltk
import textdistance
import random
from sklearn import metrics
import torch
import deepmatcher as dm
from torchtext import vocab

### Funzioni

In [6]:
#creazione delle feature dei trigrammi
def create_features(titles):
    features_vec_products = []
    
    for title_tkn in titles:
        for i, gram in enumerate(title_tkn):
            d = {}
            features_vec = []
            #distanza dall'inizio e dalla fine (0, 1)
            idx = title_tkn.index(gram)
            features_vec.append(idx)
            features_vec.append(len(title_tkn) - idx)

            # almeno una stringa è scritta in maiuscolo (2)
            flag_maiusc = 0
            for word in gram.split():
                if word.isupper():
                    features_vec.append(1)
                    flag_maiusc = 1
                    break
            if flag_maiusc == 0:
                features_vec.append(0)
                
            #tutte le stringe sono scritte in maiuscolo (3)
            flag_all_maiusc = 1
            for word in gram.split():
                if not word.isupper():
                    flag_all_maiusc = 0
                    features_vec.append(0)
                    break
            if flag_all_maiusc == 1:
                features_vec.append(1)

            #almeno una stringa contiene numeri (4)
            flag_number = 0
            for word in gram.split():
                if re.search(regex_hasNum, word):
                    features_vec.append(1)
                    flag_number = 1
                    break
            if flag_number == 0:
                features_vec.append(0)
                
            #almeno una stringa contiene solo numeri (5)
            flag_only_numbers = 0
            for word in gram.split():
                if word.isdecimal():
                    features_vec.append(1)
                    flag_only_numbers = 1
                    break
            if flag_only_numbers == 0:
                features_vec.append(0)
            
            #contiene nome del sito (6)
            flag_site = 0
            for word in gram.split():
                if word.lower() in '\t'.join(sites):
                    flag_site = 1
                    features_vec.append(1)
                    break
            if flag_site == 0:
                features_vec.append(0)

            #almeno una stringa è una specifica (7)
            flag_spec = 0
            for word in gram.split():
                for regex in specs_re:
                    if re.search(regex, word):
                        flag_spec = 1
                        break
                if flag_spec == 1:
                    features_vec.append(1)
                    break
            if flag_spec == 0:
                features_vec.append(0)

            #almeno una stringa è brand (8)
            flag_brand = 0
            for word in gram.split():
                if word.lower() in '\t'.join(brands):
                    flag_brand = 1
                    features_vec.append(1)
                    break
            if flag_brand == 0:
                features_vec.append(0)
                
            #prima stringa è brand e c'è un codice (9)
            flag_brand = 0
            if gram.split()[0].lower() in '\t'.join(brands) and (re.search(regex_code, gram.split()[1]) or re.search(regex_code, gram.split()[2])):
                features_vec.append(1)
            else:
                features_vec.append(0)

            #almeno una str codice (10)
            flag_code = 0
            for word in gram.split():
                if re.search(regex_code, word):
                    flag_code = 1
                    break
            features_vec.append(flag_code)
            
            #tutte str codice (11)
            flag_all_code = 1
            for word in gram.split():
                if not re.search(regex_code, word):
                    flag_all_code = 0
                    break
            features_vec.append(flag_all_code)

            d['gram'] = gram
            d['vector'] = features_vec

            features_vec_products.append(d)
    return features_vec_products

# estrae predizioni errate
def wrong_pred(true, predicted, idx):
    wrong_indexes = []
    for i in range(len(idx)):#df['label'].to_list()):
        if true[i] != predicted[i]:
            wrong_indexes.append(idx[i])

    return wrong_indexes

# creazione delle combinazioni dei token per i trigrammi notevoli
def create_string_combination(string, df, idx, l_or_r):
    res = []
    tok = string.split(' ')
    length = len(tok)
    for i in range(0, length):
        #print(df['{}_brand'.format(l_or_r)][idx])
        if isinstance(df['{}_brand'.format(l_or_r)][idx], str) and (df['{}_brand'.format(l_or_r)][idx] not in tok[i].lower()):
            if re.search(regex_code, tok[i]) and not [True for rgx in specs_re if re.search(rgx, tok[i])]:
                res.append(df['{}_brand'.format(l_or_r)][idx]+tok[i])
        if not tok[i].isdigit():
            res.append(tok[i])
        if i < length - 1:
            if not tok[i].isdigit() and not tok[i + 1].isdigit():
                res.append(tok[i] + tok[i+1])
    res.append(string.replace(' ', ''))
    return list(set(res))

#creazione dei trigrammi
def create_trigrams(df, l_or_r, limit): #limit=len(df)
    d = {'{}_codes'.format(l_or_r): []}
    to_ngrams = []
    for i in range(0, limit):
        if (i >= limit):
            break
        no_punc = (df['{}_page_title'.format(l_or_r)][i].replace('"', '').replace("Null", "").translate(str.maketrans('', '', string.punctuation)))
        for i in range(0,len(no_punc)):
            if (i + 1) < len(no_punc) and re.search(regex_code, no_punc[i]) and re.search(regex_code, no_punc[i + 1]):
                to_ngrams.append(no_punc[i] + no_punc[i + 1])
                i += 1
            else:
                to_ngrams.append(no_punc[i])
        tokenized = ngrams(to_ngrams.split(), 3)
        d['{}_codes'.format(l_or_r)].append([create_string_combination(' '.join(grams)) for grams in tokenized][0])
    return d

# creazione del dataset finale
def prepare_final_data(df, tok_titles, l_or_r, clf, limit=None):
    if limit == None:
        limit = len(df)
    final_list = []
    for idx, item in enumerate(tok_titles):
        if idx >= limit:
            break
        print("{}/{}\r".format(idx, limit if limit else len(tok_titles)), flush=True, end="")
        page_id = df["{}_spec_id".format(l_or_r)][idx]
        brand = df["{}_brand".format(l_or_r)][idx]
        features = create_features([item])
        df_tkn = pd.DataFrame(features)
        l = []
        codes = []
        for index, word in df_tkn.iterrows():
            d = {}
            vec = ((word['vector']))
            for i in range(0, 12):
                d['feature_' + str(i)] = vec[i]
            l.append(d)
            #df usato per svm
        df_features = pd.DataFrame(l)
        pred = clf.predict(df_features)
        for i, tag in enumerate(pred):
            if tag == 1:
                codes= codes+create_string_combination(item[i], df, idx, l_or_r)
        final_list.append({"{}_spec_id".format(l_or_r): page_id, "{}_codes".format(l_or_r): codes, "{}_brand".format(l_or_r): brand})
    return final_list

# calcolo della similarità tra due offerte per valutare il matching
def compute_score(ev_df, distance = 'jaccard', th=0.9):
    indexes = ev_df[0]
    left_codes = ev_df[2]
    left_brand = ev_df[3]
    right_codes = ev_df[5]
    right_brand = ev_df[6]
    
    predicted_nb = []
    idx_to_dm = []
    score = []
    comp = []
    empty_idx_list = []
    
    for i in range(0, len(indexes)):
        print(i, end='\r', flush=True)
        if not isinstance(left_brand[i], float) and not isinstance(right_brand[i], float) and (left_brand[i].upper() != right_brand[i].upper()):
            predicted_nb.append(0)
            score.append(0)
            comp.append('EXLUDED BY BRANDS: {} ------ {}'.format(left_brand[i], right_brand[i]))
            continue
        if not left_codes[i] or not right_codes[i]:
            predicted_nb.append(0)
            idx_to_dm.append(indexes[i])
            score.append(0)
            comp.append('TO_DM')
            empty_idx_list.append(i)
            continue
        max_score = 0
        match = 0
        comparisons = ''
        left_target = None
        right_target = None
        for item in left_codes[i]:
            for item_r in right_codes[i]:
                if distance == 'lev':
                    match_score = lv.ratio(item.upper(), item_r.upper())
                elif distance == 'jaccard':
                    #match_score = 1 - nltk.jaccard_distance(set(item), set(item_r))
                    match_score = textdistance.jaccard.similarity(item.upper(), item_r.upper())
                elif distance == 'jw':
                    match_score = textdistance.jaro_winkler.similarity(item.upper(), item_r.upper())
                elif distance == 'cosine':
                    match_score = textdistance.cosine.similarity(item.upper(), item_r.upper())
                else:
                    return []
                if match_score > max_score and re.search(regex_code, item) and re.search(regex_code, item_r) and item.lower() not in '\t'.join(sites) and item_r.lower() not in '\t'.join(sites) and len([True for reg in specs_re if re.search(reg, item)]) == 0 and len([True for reg in specs_re if re.search(reg, item_r)]) == 0:
                    max_score = match_score
                    left_target = item
                    right_target = item_r
                    comparisons = left_target + '---> ' + right_target + ' -----SCORE: ' + str(max_score) + ' ------IDX' + str(indexes[i])
        if max_score > th:
            match = 1
        predicted_nb.append(match)
        score.append(max_score)
        comp.append(comparisons)
    return [predicted_nb, idx_to_dm, score, comp, empty_idx_list]

### Input variables

In [7]:
# percorsi all'elenco dei brand elettronici e dei siti di eCommerce
brands_path = './utils/electronic_brands.txt'
sites_path = './utils/sites.txt'

# percorso al dataset di input
dataset_path = './data/dataset_monitor.csv'

# percorso al dataset di training per l'estrazione di trigrammi notevoli
product_recognition_training_set = './data/product_recognition_train.csv'

# precorso al modello allenato di DeepMatcher per la classificazione del matching
deepMatcher_model_path = './data/deepMatcher_model.pth'

# sample del dataset da valutare
n_sample = 30000

# soglia similarità [0, 1]
threshold = 0.9

In [8]:
f = open(brands_path, 'r')
brands = f.read().splitlines()
f.close()
f = open(sites_path, 'r')
sites = f.read().splitlines()
f.close()

In [9]:
# dataframe iniziale
df = pd.read_csv(dataset_path)
columns = ['left_spec_id', 'right_spec_id', 'left_page_title', 'right_page_title', 'left_brand', 'right_brand', 'label']
df = df[columns]

In [10]:
# codice
regex_code = re.compile('^(?=.*[a-zA-Z])(?=.*[0-9])')

#caratteristiche tecniche
regex_px = re.compile('\d+[xX]\d+$')
regex_mem = re.compile('\d+(MB|GB|TB|mb|gb|tb|Mb|Gb|Tb|mB|gB|tB){1}$')
regex_eia = re.compile('RS\d+$')
regex_proc = re.compile('(i\d$|i\d-)')
regex_inch = re.compile('(inch|Inch|INCH|in|IN|In)')
regex_freq = re.compile('(Ghz|GHZ|ghz|GHz|Hz|HZ|hz)')
regex_ordinal = re.compile('(1st|2nd|3rd|\dth$)')
regex_bit = re.compile('(\d+-bit|\d+Bit|\d+BIT|\d+-Bit|\d+-BIT)')
regex_hasNum = re.compile('\d')

# lista delle regex sulle caratteristiche tecniche
specs_re = [regex_px, regex_mem, regex_eia, regex_freq, regex_inch, regex_mem, regex_ordinal, regex_proc, regex_px]

### Training SVM e MLP per Product Recognition

In [15]:
df_train = pd.read_csv(product_recognition_training_set, sep=';')

In [16]:
l = []
for index, row in df_train.iterrows():
    d = {}
    vec = (json.loads(row['vector']))
    for i in range(0, 12):
        d['feature_' + str(i)] = vec[i]
    d['label'] = row['code']
    l.append(d)

#df usato per allenare classificatori
df_features = pd.DataFrame(l)

In [17]:
# Import train_test_split function
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df_features.drop('label', 1, inplace=False), df_features['label'], test_size=0.1,random_state=109) # 90% training and 10% test

X = pd.concat([X_train, y_train], axis=1)

not_code = X[X.label==0]
code = X[X.label==1]
code_upsampled = resample(code,
                          replace=True, # sample with replacement
                          n_samples=len(not_code), # match number in majority class
                          random_state=27) # reproducible results

upsampled = pd.concat([not_code, code_upsampled])

X_train = upsampled.drop('label', axis=1)
y_train = upsampled.label

In [18]:
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

#Trainining SVM con ottimizzazione iperparametri
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
clf_svm = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=10)
clf_svm.fit(X_train, y_train)
print('Best parameters found:\n', clf_svm.best_params_)
print(clf_svm)

#Predict the response for test dataset
y_pred_svm = clf_svm.predict(X_test)

Best parameters found:
 {'C': 10, 'gamma': 1}
GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'gamma': [0.001, 0.01, 0.1, 1]})


In [19]:
#Trainining MLP con ottimizzazione iperparametri
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf_mlp = GridSearchCV(MLPClassifier(max_iter=100), parameter_space, n_jobs=-1, cv=3)
clf_mlp.fit(X_train, y_train)
print('Best parameters found:\n', clf_mlp.best_params_)

y_pred_mlp = clf_mlp.predict(X_test)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}


/Users/aparo/jupyter/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


### Conversione in trigrammi di tutti i titoli

In [20]:
left_titles = list(df['left_page_title'])
left_titles_tokenized = []
for item in left_titles:
    no_punc = (item.replace('"', '').replace("Null", "").translate(str.maketrans('', '', string.punctuation)))
    tokenized = ngrams(no_punc.split(), 3)
    left_titles_tokenized.append([' '.join(grams) for grams in tokenized])

right_titles = list(df['right_page_title'])
right_titles_tokenized = []
for item in right_titles:
    no_punc = (item.replace('"', '').replace("Null", "").translate(str.maketrans('', '', string.punctuation)))
    tokenized = ngrams(no_punc.split(), 3)
    right_titles_tokenized.append([' '.join(grams) for grams in tokenized])

In [22]:
#df_final_left = pd.DataFrame(left[0])
#df_final_right = pd.DataFrame(right[0])
final_list_left = prepare_final_data(df, left_titles_tokenized,'left', clf_svm)
final_list_right = prepare_final_data(df, right_titles_tokenized,'right', clf_svm)
df_final_left = pd.DataFrame(final_list_left)
df_final_right = pd.DataFrame(final_list_right)

df_final = pd.concat([df_final_left,df_final_right],axis=1)

In [23]:
# sample randomico del df in input
def sample_df(df, n_sample):
    if n_sample >= len(df):
        return df
    return df.sample(n=n_sample).reset_index()

# creazione della lista in input alla funzione che calcola la similarità tra due offerte
def extract_columns(df):
    return [df['index'].tolist(), df['left_spec_id'].tolist(), df['left_codes'].tolist(), df['left_brand'].tolist(), df['right_spec_id'].tolist(), df['right_codes'].tolist(), df['right_brand'].tolist()]

In [24]:
df_to_eval = sample_df(df_final, n_sample)
eval_input = extract_columns(df_to_eval)

### Valutazione

In [25]:
code_based_output = compute_score(eval_input, threshold)

In [26]:
# calcolo dei confronti tra offerte valutati con deepMatcher
def compute_combined(df, cb_output, model_path):
    try:
        vocab._default_unk_index
    except AttributeError:
        def _default_unk_index():
            return 0
        vocab._default_unk_index = _default_unk_index
    
    output = cb_output
    
    index_list = output[1]
    to_dm = df.iloc[index_list]
    to_dm['id'] = [num for num in range(0, len(to_dm))]
    del to_dm['label']
    to_dm.to_csv('./unlabeled_to_dm.csv', index=False)
    model = dm.MatchingModel()
    model.load_state(model_path)
    unlabeled_proc = dm.data.process_unlabeled(path='./unlabeled_to_dm.csv', trained_model=model)
    predictions = model.run_prediction(unlabeled_proc, output_attributes=True)
    pred_output = [0 if score < 0.9 else 1 for score in predictions['match_score']]
    dm_df = pd.DataFrame(data=pred_output, columns=["label"])
    
    labels = dm_df['label']
    for i, idx in enumerate(output[4]):
        output[0][idx] = labels[i]
    
    return output
    
    

In [27]:
combined_output = compute_combined(df, code_based_output, deepMatcher_model_path)

<ipython-input-26-4c5b7b638b2a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_dm['id'] = [num for num in range(0, len(to_dm))]
/Users/aparo/jupyter/lib/python3.9/site-packages/torchtext/data/field.py:150: UserWarning: MatchingField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)

Reading and processing data from "./unlabeled_to_dm.csv"
/Users/aparo/jupyter/lib/python3.9/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torc

===>  PREDICT Epoch 4


/Users/aparo/jupyter/lib/python3.9/site-packages/torchtext/data/iterator.py:48: UserWarning: MatchingIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/aparo/jupyter/lib/python3.9/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 4 || Run Time:    5.4 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



### Valutazione score

In [28]:
y_true = df.loc[eval_input[0]]['label'].tolist()

print("Precision:",metrics.precision_score(y_true, combined_output[0]))
print("Recall:",metrics.recall_score(y_true, combined_output[0]))
print("f1-score:", metrics.f1_score(y_true, combined_output[0]))

Precision: 1.0
Recall: 0.782
f1-score: 0.877665544332211
